In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from datasets import load_dataset, load_dataset_builder

In [3]:
import sys
sys.path.append('../..')

from baselines.galaxy10_decals.dataset_wrapper import DatasetWrapper

In [4]:
dataset_loc = 'mwalmsley/galaxy10_decals'

In [5]:
# Load Dataset Builders 
decals10_builder = load_dataset_builder(dataset_loc, 'astropile', trust_remote_code=True)
# decals10_builder.set_format('torch')

In [6]:
type(decals10_builder)

datasets_modules.datasets.mwalmsley--galaxy10_decals.f7f0215e9b20abfd8f74854d9ac987a9e7ef308f4a8681fa7840260fba92924c.galaxy10_decals.GZDecals

In [7]:
decals10_train = load_dataset('mwalmsley/galaxy10_decals', 'astropile', split='train', trust_remote_code=True)
decals10_test = load_dataset('mwalmsley/galaxy10_decals', 'astropile', split='test', trust_remote_code=True)

In [8]:

decals10_train.set_format('torch')
decals10_test.set_format('torch')

In [9]:
len(decals10_train)

200

In [10]:
decals10_train[0]['label']

tensor(5)

In [11]:
decals10_train[0]['images_by_band']['band']

['g', 'r', 'z']

In [12]:
decals10_train[0]['images_by_band']['array'].shape

torch.Size([3, 224, 224])

In [13]:
type(decals10_train)

datasets.arrow_dataset.Dataset

In [14]:
wrapper = DatasetWrapper(
    train_dataset=decals10_train,
    test_dataset=decals10_test,
    feature_flag='images_by_band.array',
    label_flag='label',
    loading='iterated'
)

In [15]:
# decals10_train['images_by_band']

In [18]:
wrapper.prepare_data()
wrapper.feature_mean, wrapper.feature_std

(tensor([[[0.0065]],
 
         [[0.0129]],
 
         [[0.0205]]]),
 tensor([[[0.0664]],
 
         [[0.1309]],
 
         [[0.1783]]]))

In [19]:
train_loader = wrapper.train_dataloader()
val_loader = wrapper.val_dataloader()
test_loader = wrapper.test_dataloader()

In [20]:
for images, labels in train_loader:
    print(images.shape, labels.shape)

torch.Size([128, 3, 224, 224]) torch.Size([128])
torch.Size([32, 3, 224, 224]) torch.Size([32])


In [21]:
im_np = images[0].numpy().transpose(1, 2, 0)
im_np.min(), im_np.max()

(-0.11923878, 2.390703)

In [27]:
import galaxy10_model

config = galaxy10_model.default_config()


model = galaxy10_model.SmallConvModel(config)

In [38]:
trainer = pl.Trainer(max_epochs=1, accelerator='gpu', logger=False, enable_checkpointing=False)


trainer.fit(model, wrapper)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | encoder        | Sequential         | 19.4 K
1 | head           | Sequential         | 1.4 M 
2 | model          | Sequential         | 1.4 M 
3 | train_accuracy | MulticlassAccuracy | 0     
4 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.618     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/walml/miniforge3/envs/zoobot39_dev/lib/python3.9/site-packages/pytorch_lightning/core/module.py:491: You called `self.log('val_acc', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Training: |          | 0/? [00:00<?, ?it/s]

In [29]:
# alternative non-lightning loading option

In [30]:
# target_size = 64
# from torchvision.transforms import Compose, Normalize, Resize, Lambda, RandomHorizontalFlip, RandomVerticalFlip


# normalize = Normalize(mean=1., std=1.)
#                     #   https://pytorch.org/vision/main/generated/torchvision.transforms.Lambda.html)

# _transforms = Compose(
#     [
#         Resize(target_size),
#         RandomHorizontalFlip(),
#         RandomVerticalFlip(),
#         Lambda(func=lambda x: np.arcsinh(x)),
#         normalize
#     ]
# )

# def transforms(examples):
#     examples["pixel_values"] = [_transforms(image.convert("RGB")) for image in examples["image"]]
#     return examples